# Import RPB model along with other utility functions

In [ ]:
from RPB_model import *

# Creating model for a single side/section of the RPB

In [ ]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the boundary conditions for each case.
# m=RPB_model(mode="adsorption", gas_flow_direction=1)
# or
m=RPB_model(mode="desorption", gas_flow_direction=-1)

# Custom initialization routine. Uses a homotopy routine to adjust the initialization factors so that all equations are 'active'.
single_section_init(m)
# homotopy_init_routine(m)

In [ ]:
# Some various utility functions that I have been using to check model performance
evaluate_MB_error(m)

print(f'CO2 Capture = {m.CO2_capture():.3}')

In [ ]:
check_scaling(m)

In [ ]:
jac, variables, constraints = scaling_script(m)

In [ ]:
plotting(m)

In [ ]:
print(jac.getcol(11749))

In [ ]:
variables[11749].pprint()

In [ ]:
constraints[8377].pprint()

In [ ]:
constraints[12768].pprint()

In [ ]:
constraints[3836].pprint()

In [ ]:
constraints[2496].pprint()

In [ ]:
variables[461].pprint()

In [ ]:
variables[768].pprint()

In [ ]:
[m.Rs_CO2[z,o]() for z,o in m.dheat_fluxdz.keys()]

In [ ]:
print(iscale.get_scaling_factor(constraints[927]))

# Creating a full RPB model

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

In [ ]:
# initialize using BlockTriangularizationInitializer() with a list of values for initialization factors within the models
init_routine_1(RPB, homotopy_steps=[0.1, 1])

In [ ]:
# or load a previous solution and solve
from_json(RPB, fname="opt case 122023.json.gz", gz=True)

In [ ]:
# or load a previous solution and solve
from_json(RPB, fname="opt case 122023.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

In [ ]:
report(RPB)

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
check_scaling(RPB)

In [ ]:
jac, variables, constraints = scaling_script(RPB)

In [ ]:
print(jac.getcol(15066))

In [ ]:
constraints[12301].pprint()

In [ ]:
constraints[3964].pprint()

In [ ]:
variables[1760].pprint()

In [ ]:
variables[1739].pprint()

In [ ]:
variables[7419].pprint()

In [ ]:
print(iscale.get_scaling_factor(constraints[2722]))

In [ ]:
print(iscale.get_scaling_factor(variables[4336]))

In [ ]:
[RPB.ads.dTsdo[z,o]() for z,o in RPB.ads.dTsdo.keys()]

# Optimization

In [ ]:
# change design to fix capture by freeing up inlet adsorber pressure

RPB.ads.P_in.unfix()
RPB.ads.CO2_capture.fix(0.95)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

# or load a previous solution and solve
from_json(RPB, fname="opt case 010324 95perc capture.json.gz", gz=True)

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
report(RPB)

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

In [ ]:
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
# RPB.des.Tx.fix(160+273)
# RPB.des.Tx.pprint()

In [ ]:
RPB.ads.P_in.setub(4)
RPB.ads.P_in.pprint()

In [ ]:
RPB.ads.P.setub(4)

In [ ]:
RPB.ads.C_tot.setub(200)

In [ ]:
RPB.des.P_in.setub(3)
RPB.des.P_in.setlb(1.01325)
RPB.des.P_in.pprint()

In [ ]:
RPB.des.P.setub(3)

In [ ]:
RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(5)
RPB.ads.w_rpm.pprint()

In [ ]:
[RPB.ads.dPdz[z,o].setlb(-5) for z,o in RPB.ads.dPdz.keys()]

In [ ]:
[RPB.des.dPdz[z,o].setub(3) for z,o in RPB.des.dPdz.keys()]

In [ ]:
# free up decision variables (keep des.Tx fixed for first run the free up later)
RPB.ads.L.unfix()
RPB.ads.theta.unfix()
RPB.des.P_in.unfix()
RPB.ads.Tx.unfix()
RPB.des.Tx.unfix()
RPB.ads.w_rpm.unfix()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
RPB.alpha_obj = 0.7

In [ ]:
RPB.alpha_obj = 0.999

In [ ]:
RPB.alpha_obj.display()

In [ ]:
RPB.ads.CO2_capture.fix(0.94)
# RPB.ads.CO2_capture.setlb(0.90)

In [ ]:
# solve using conopt thorugh gams
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
# or solve using ipopt
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-8,
    # "halt_on_ampl_error": "yes",
    "tol": 1e-4,
    "max_cpu_time": 5*60,
    # "mu_strategy": "adaptive",
}
solver.solve(RPB, tee=True).write()

In [ ]:
results_df = report(RPB)
results_df

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
degen_hunter(RPB)

# Sensitivity Testing

In [ ]:
RPB.objective = Objective(expr=0)

In [ ]:
RPB.ads.CO2_capture=0.9

In [ ]:
RPB.ads.theta.fix()

In [ ]:
RPB.ads.theta = 0.7
RPB.des.theta = 0.3

In [ ]:
RPB.ads.w_rpm = 0.0005

In [ ]:
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
report(RPB)

In [ ]:
[RPB.ads.qCO2[z,o]() for z,o in RPB.ads.qCO2_eq.keys()]

In [ ]:
RPB.alpha_obj = 0.999

In [ ]:
RPB.alpha_obj()

In [ ]:
RPB.ads.a_sp = 30
RPB.des.a_sp = 30

In [ ]:
alpha_list=[
    0.0001,
    0.001,
    0.005,
    0.01,
    0.02,
    0.05,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    0.925,
    0.95,
    0.975,
    0.99,
    0.999,
]

In [ ]:
alpha_list = alpha_list[::-1]

In [ ]:
E = []
P = []

for j in alpha_list:
    RPB.alpha_obj = j

    results = SolverFactory("gams").solve(
        RPB,
        tee=True,
        keepfiles=True,
        solver="conopt4",
        tmpdir="temp",
        add_options=["gams_model.optfile=1;"],
    )

    print(f'alpha = {j}, E={RPB.energy_requirement()}, P={RPB.productivity()}')

    E.append(RPB.energy_requirement())
    P.append(RPB.productivity())


In [ ]:
pd.DataFrame({'alpha':alpha_list,'E':E,'P':P})

In [ ]:
plt.scatter(E,P)

In [ ]:
RPB.alpha_obj.pprint()

In [ ]:
alpha_list2=[0.96]

for j in alpha_list2:
    RPB.alpha_obj = j

    results = SolverFactory("gams").solve(
        RPB,
        tee=True,
        keepfiles=True,
        solver="conopt4",
        tmpdir="temp",
        add_options=["gams_model.optfile=1;"],
    )

    E.append(RPB.energy_requirement())
    P.append(RPB.productivity())
    alpha_list.append(j)

In [ ]:
report(RPB)

# Polishing step optimization

In [ ]:
from RPB_model import *

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

# or load a previous solution and solve
from_json(RPB, fname="polishing step opt case 010424 95perc capture max prod.json.gz", gz=True)

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

In [ ]:
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
# RPB.des.Tx.fix(160+273)
# RPB.des.Tx.pprint()

In [ ]:
RPB.ads.P_in.setub(10)
RPB.ads.P_in.pprint()

In [ ]:
RPB.ads.P.setub(10)

In [ ]:
RPB.ads.C_tot.setub(500)

In [ ]:
RPB.des.P_in.setub(3)
RPB.des.P_in.setlb(1.01325)
RPB.des.P_in.pprint()

In [ ]:
RPB.des.P.setub(3)

In [ ]:
RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(5)
RPB.ads.w_rpm.pprint()

In [ ]:
[RPB.ads.dPdz[z,o].setlb(-20) for z,o in RPB.ads.dPdz.keys()]

In [ ]:
[RPB.des.dPdz[z,o].setub(10) for z,o in RPB.des.dPdz.keys()]

In [ ]:
RPB.ads.vel.setub(10)
RPB.des.vel.setub(10)

In [ ]:
# free up decision variables (keep des.Tx fixed for first run the free up later)
RPB.ads.L.unfix()
RPB.ads.theta.unfix()
RPB.des.P_in.unfix()
RPB.ads.Tx.unfix()
RPB.des.Tx.unfix()
RPB.ads.w_rpm.unfix()

In [ ]:
RPB.alpha_obj = 0.8

In [ ]:
# RPB.ads.CO2_capture.fix(0.975)

In [ ]:
# solve using conopt thorugh gams
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
results_df = report(RPB)
results_df

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
degen_hunter(RPB)

In [ ]:
RPB.ads.y_in.display()

In [ ]:
RPB.ads.y_in["CO2"] = 0.004149
RPB.ads.y_in["N2"] = 0.902851
RPB.ads.y_in["H2O"] = 0.093

In [ ]:
new = 0.04
RPB.ads.y_in["CO2"] = new
RPB.ads.y_in["H2O"] = 0.09
RPB.ads.y_in["N2"] = 1-new-0.09

In [ ]:
RPB.ads.y_in["CO2"] = 0.04
RPB.ads.y_in["N2"] = 0.87
RPB.ads.y_in["H2O"] = 0.09

# Plotting

In [ ]:
full_contactor_plotting(RPB)

# Save Model

In [ ]:
# save model
to_json(RPB, fname="PCC opt case 010424 90perc capture max prod.json.gz", gz=True, human_read=False)

In [ ]:
tuple(np.geomspace(0.005, 0.1, 5)) + tuple(np.linspace(0.1, 0.995, 5)[1:])

In [ ]:
tuple(np.linspace(0.5, 0.995, 5)[1:])